# 1 задание

In [66]:
import random
import numpy as np
num = 50000
def simulate_reed_sequencing():
    alphabet = ['A', 'T', 'G', 'C']
    dna = ''.join(random.choices(alphabet, k=num))
    reads = []
    ground_truths = []
    for i in range(num):
        read_length = int(np.random.normal(250, 30))
        start = random.randint(0, len(dna) - read_length)
        read = dna[start:start+read_length]
        ground_truths.append(read)

        errors = []
        p_values = []
        for j in range(read_length):
            N = random.randint(0, 60)
            if N > 0:
                true_nucleotide = read[j]
                other_nucleotides = [n for n in ['A', 'T', 'G', 'C'] if n != true_nucleotide]
                read = read[:j] + random.choice(other_nucleotides) + read[j+1:]
                errors.append((j, true_nucleotide))
                p_values.append(0.1)
            else:
                p_values.append(0.01)

        p = len(errors) / read_length
        reads.append((read, errors, p_values))

    with open('reads.fastq', 'w') as f:
        for i, (read, errors, p_values) in enumerate(reads):
            f.write(f'@READ_{i}\n')
            f.write(f'{read}\n')
            f.write('+\n')
            qualities = []
            for j in range(len(read)):
                p = p_values[j]
                quality = -10 * np.log10(p)
                qualities.append(chr(int(quality) + 33))
            f.write(''.join(qualities) + '\n')

    return ground_truths

# 2 задание

In [76]:
import subprocess
from Bio import SeqIO


def evaluate_trimmomatic_performance(ground_truths):
    path = 'D:\\Study\\BioinformaticsCourse2023\\2_1\\Trimmomatic-0.39\\trimmomatic-0.39.jar'
    trim_command = f'java -jar {path} SE -phred33 reads.fastq output.fastq ILLUMINACLIP:TruSeq3-SE:2:30:10 TRAILING:3'

    result = subprocess.run(trim_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


    trimmed_reads = list(SeqIO.parse('output.fastq', 'fastq'))

    correct_removals = 0
    incorrect_removals = 0
    for i in range(len(trimmed_reads)):
        trimmed_read = str(trimmed_reads[i].seq)
        ground_truth_read = ground_truths[i]
        if len(trimmed_read) < len(ground_truth_read):
            for j in range(len(trimmed_read)):
                if trimmed_read[j] != ground_truth_read[j]:
                    incorrect_removals += 1
            correct_removals += len(ground_truth_read) - len(trimmed_read)
        else:
            for j in range(len(ground_truth_read)):
                if trimmed_read[j] != ground_truth_read[j]:
                    incorrect_removals += 1

    print(f'Correct removals: {correct_removals}')
    print(f'Incorrect removals: {incorrect_removals}')

ground_truths = simulate_reed_sequencing()
evaluate_trimmomatic_performance(ground_truths)

Correct removals: 0
Incorrect removals: 12267191


# 3 задание

Слишком много некорректно убранных нуклеотидов. Попробуем Pollux

In [78]:
def compare_reads(original_file, corrected_file):

    tp = tn = fp = fn = 0


    with open(original_file) as original, open(corrected_file) as corrected:
        while True:
            original_id = original.readline().strip()
            original_seq = original.readline().strip()
            original_plus = original.readline().strip()
            original_qual = original.readline().strip()

            corrected_id = corrected.readline().strip()
            corrected_seq = corrected.readline().strip()
            corrected_plus = corrected.readline().strip()
            corrected_qual = corrected.readline().strip()
            # Stop if we've reached the end of either file
            if not original_id or not corrected_id:
                break

            for original_base, corrected_base in zip(original_seq, corrected_seq):
                if original_base == corrected_base:
                    tn += 1
                else:
                    tp += 1

    return tp, tn, fp, fn

tp, tn, fp, fn = compare_reads('reads.fastq', 'pollux-master/output/output.fastq.low')
print(f"TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

TP: 0, TN: 12471751, FP: 0, FN: 0


Вообще он должен был исправить ошибки, но почему-то нет. Возможно изначально стоит поменять настройки trimmomatic, чтобы он не так часто ошибался, тогда наверное результат должен быть лучше. У нас изначально очень низка ошибка при "секвинировании", возможно и это влияет.